## Collaborative Filters Via Gaussian Mixtures

Our goal will be to complete entries in a matrix. The dataset used will be a sample of netflix users' ratings. Logically, each user only rates a minimal portion of the full content, resulting in a sparse matrix where most of its entries are not defined which we shall treat as 0s.

To predict user ratings, we shall use gaussian mixtures models (GMMs), and to find such GMMs, the EM algorithm.

### Expectation Maximization Algorithm

The EM algorithm solves for GMMs where a sample is assumed to be generated by several normal distributions.
For this particular case, we shall assume spherical normal distributions (the covariance matrix is diagonal). Thus being generated from  $\mathcal{N}\left(\mathbf x^{(i)}; \mu ^{(j)},\sigma _ j^2 I\right)$

The EM algorithm works by maximizing the log likelihood of the mixture, with respect to the parameter set $\theta = \left\{ p_1, \dots , p_ K, \mu ^{(1)}, \dots , \mu ^{(K)}, \sigma _1^2, \dots , \sigma _ K^2\right\}$. There is no closed form solution in the regime of GMM, and therefore EM is an iterative way to find a local optimal $\theta$.


#### E-Step

There are 2 steps in the algorithm. The first one involves calculating the posterior probability that point $\mathbf x^{(i)}$ was generated by cluster, for every $i = 1,...,n$ and $j=1,...,K$. It is assumed that $\theta$ is known.
The posterior is defined as below:

$$\displaystyle  p(\text {point }\mathbf x^{(i)}\text { was generated by cluster }j | \mathbf x^{(i)}, \theta ) \triangleq p(j \mid i) = \frac{p_ j \mathcal{N}\left(\mathbf x^{(i)}; \mu ^{(j)},\sigma _ j^2 I\right)}{p(\mathbf x^{(i)} \mid \theta )}$$.

The result is that every point gets a soft count (a probability) for belonging to each cluster. This contrasts with K-means where each point is assigned to a unique cluster.

#### M-Step

The second step involves maximizing a proxy function (as defined below) over $\theta$

$$\displaystyle  \hat{\ell }(\mathbf x^{(1)},\dots ,\mathbf x^{(n)} \mid \theta ) \triangleq \sum _{i=1}^{n} \sum _{j = 1}^ K p(j \mid i) \log \left( \frac{p\left( \mathbf x^{(i)} \text { and } \mathbf x^{(i)} \text { generated by cluster }j \mid \theta \right)}{p(j \mid i)} \right).$$

Taking derivatives w.r.t. to each variable, setting them to 0 and solving we find that the optimizers are

$$\displaystyle  \widehat{\mu ^{(j)}} = \frac{\sum _{i = 1}^ n p (j \mid i) \mathbf x^{(i)}}{\sum _{i=1}^ n p(j \mid i)}$$

$$\displaystyle \widehat{p_ j} = \frac{1}{n}\sum _{i = 1}^ n p(j \mid i),$$

$$\displaystyle \widehat{\sigma _ j^2} = \frac{\sum _{i = 1}^ n p(j \mid i) \|  \mathbf x^{(i)} - \widehat{\mu ^{(j)}} \| ^2}{d \sum _{i = 1}^ n p(j \mid i)}.$$

#### Initialization

Initialization  can be done either randomly or by finding the cluster centers through K-means and using the global variance of the dataset as the variance of each cluster. In such case, the proportion counts assigned by K-means can be used as the mixtures' weights.

#### Missing Entries

Finally, because we have missing values, we must focus on the marginal distribution of the observed values alone. Since we are under the regime of spherical, independent normal distributions, the mixture density can be written as:

$$P(x_{C_ u}^{(u)} | \theta ) = \sum ^{K}_{j=1} \pi _ j N(x_{C_ u}^{(u)}; \mu _{C_ u}^{(j)}, \sigma _ j^{2}I_{|C_ u| \times |C_ u|})$$

And we shall focus on the log domain for the E-step

$$\displaystyle  \ell (j|u) = f(u,j)-\log \left(\sum _{j=1}^{K}\exp (f(u,j))\right)$$

Where $f(u,i)=\log (\pi _{i})+\log \left(N(x_{C_{u}}^{(u)};\mu _{C_{u}}^{(i)},\sigma _{i}^{2}I_{C_{u}\times C_{u}})\right)$, $x_{C_{u}}^{(u)}=\{ x_{i}^{(u)}:\,  i\in C_{u}\}$ is the set of observed values,  $\mu _{C_ u}^{(j)}$ is the equivalent for the mean, and $I_{|C_ u| \times |C_ u|})$ is the identity matrix in $|C_ u|$ dimensions.

Optimizing for our parameters of interest we end up with:
$$\displaystyle  \widehat{\mu _ l^{(k)}} = \frac{\sum _{u = 1}^ n p(k \mid u) \delta (l,C_ u) x_ l^{(u)}}{\sum _{u=1}^ n p(k \mid u) \delta (l,C_ u)},$$

Where $\delta (i,C_ u) = 1$ if $i \in C_ u$ and $\delta (i,C_ u) = 0$ if $i \notin C_ u$.

$$\displaystyle  \widehat{\sigma _ k^2} = \frac{1}{\sum _{u=1}^ n |C_ u| p(k \mid u)} \sum _{u = 1}^ n p(k \mid u) \| x_{C_ u}^{(u)} - \widehat{\mu _{C_ u}^{(k)}}\| ^2,$$

$$\displaystyle \widehat{\pi _ k} = \frac{1}{n} \sum _{u=1}^ n p(k \mid u).$$

(We let go of the proof in this step for conciseness).

In [2]:
from typing import Tuple
import numpy as np
from scipy.special import logsumexp
from common import *  # Functions in common were provided by assignment statement! - except for bic
import numpy as np
import pdb

In [172]:
def estep(X: np.ndarray, mixture: GaussianMixture) -> Tuple[np.ndarray, float]:
    """E-step: Softly assigns each datapoint to a gaussian component

    Args:
        X: (n, d) array holding the data, with incomplete entries (set to 0)
        mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples
        float: log-likelihood of the assignment

    """
    n , K = X.shape[0] , mixture.var.shape[0]
    post = np.zeros((n,K))
    LL=0    # Log-likelihood
    for i in range (n):
        mask = X[i] != 0  # Mask to select observed values only
                     
        mu , var , P = mixture.mu[:,mask] , np.expand_dims(mixture.var,1) , np.expand_dims(mixture.p,1) 
        d = mu.shape[1]
        
        XX = np.tile(X[i][mask], (K, 1))  # np.tile is used to allow for calculations in matrix form
        den = (-d/2)*np.log(2*np.pi*var)
        exp = -np.linalg.norm(XX-mu,axis=1,keepdims=2)**2/(2*var)
        l_norm = den+exp
        f = np.log(P+1e-16) + l_norm  # 1e-16 added for numerical stability, f as f(u,i)
        LL += logsumexp(f)
        post[i,:]= np.exp((f - np.tile(logsumexp(f),(K,1))).T)
        

    return (post,LL)


def mstep(X: np.ndarray, post: np.ndarray, mixture: GaussianMixture,
          min_variance: float = .25) -> GaussianMixture:
    """M-step: Updates the gaussian mixture by maximizing the log-likelihood
    of the weighted dataset

    Args:
        X: (n, d) array holding the data, with incomplete entries (set to 0)
        post: (n, K) array holding the soft counts
            for all components for all examples
        mixture: the current gaussian mixture
        min_variance: the minimum variance for each gaussian

    Returns:
        GaussianMixture: the new gaussian mixture
    """
    n , K = X.shape[0] , post.shape[1]
    mu , var , P = mixture.mu , mixture.var , mixture.p
    
    N = post.sum(axis = 0)
    P = N/n

    for j in range (K):
        d = X.shape[1]
        ind = X.copy()
        ind[ind!=0] = 1.0  # indicator function
        
        mu_hat = np.sum(post[:,[j]]*X,axis = 0)/np.sum(ind*post[:,[j]],axis = 0)
        mask = np.sum(post[:,[j]]*ind,axis=0) >= 1  # Mask to update mu when an entire point is assigned to a cluster
        mu_hat_j = mu[j].copy()
        mu_hat_j[mask] = mu_hat[mask]
        mu[j] = mu_hat_j
        
        d_Cu = np.sum(ind, axis = 1,keepdims=1)  # Dimension of set of observed points
        denom = np.sum(d_Cu*post[:,[j]])  # Denominator for variance update
        mutile=np.tile(mu[j],(n,1))*ind
        sq_dist = np.linalg.norm(X - mutile, axis = 1,keepdims = 1)**2
        var[j] = (post[:,[j]]*sq_dist).sum(axis= 0)/denom
    var [ var < min_variance ] = min_variance
    
    return (GaussianMixture(mu, var, P))
    

def fill_matrix(X: np.ndarray, mixture: GaussianMixture) -> np.ndarray:
    """Fills an incomplete matrix according to a mixture model

    Args:
        X: (n, d) array of incomplete data (incomplete entries =0)
        mixture: a mixture of gaussians

    Returns
        np.ndarray: a (n, d) array with completed data
    """

    XX = X.copy()    
    post = estep(X,mixture)[0]
    mask = X == 0
    
    E_matrix = post @ mixture.mu  # Matrix with expected values for all points
    XX[mask] = E_matrix[mask]  # Update where there are missing values
    return XX

def run(X: np.ndarray, mixture: GaussianMixture,
            post: np.ndarray) -> Tuple[GaussianMixture, np.ndarray, float]:
        """Runs the mixture model
    
        Args:
            X: (n, d) array holding the data
            post: (n, K) array holding the soft counts
                for all components for all examples
    
        Returns:
            GaussianMixture: the new gaussian mixture
            np.ndarray: (n, K) array holding the soft counts
                for all components for all examples
            float: log-likelihood of the current assignment
        """
        old = None
        new = None
        while old is None or new - old > abs(new)*1e-6:
            
            old=new
            post , new = estep(X,mixture)
            mixture = mstep(X,post,mixture)         

        return (mixture, post, new)
    
def super_run(X,K,n_seeds):
        """Executes several runs with different seed values
    
        Args:
            X: (n, d) array holding the data
            K: int, number of clusters
            n_seeds: integer, used as range in for loop
    
        Returns:
            GaussianMixture: best performing GaussianMixture
            LL: float, log likelihood with best GaussianMixture
        """
        LL = []
        GM = []
        for seed in range (n_seeds):

            X_init = init(X,K,seed)
            run_result = run(X,X_init[0],X_init[1]) 
            GM += [run_result[0]]
            LL += [run_result[2]] #[seed]
            print(f'LL for K = {K} and seed = {seed}, \n{LL[seed]}')
        maxLL = max(LL) ; best_GM = GM[LL.index(maxLL)]
        return best_GM,maxLL

With our algorithm ready to calculate the optimal $\theta$ we can now proceed to create the rest of the functions we need to predict values.
Fill matrix will replace missing values (0s) with expected values.
Run makes a full run of the algorithm, returning a gaussian mixture, the soft counts and the log likelihood. Super run iterates over a user defined number of seeds.

In [ ]:

def fill_matrix(X: np.ndarray, mixture: GaussianMixture) -> np.ndarray:
    """Fills an incomplete matrix according to a mixture model

    Args:
        X: (n, d) array of incomplete data (incomplete entries =0)
        mixture: a mixture of gaussians

    Returns
        np.ndarray: a (n, d) array with completed data
    """

    XX = X.copy()    
    post = estep(X,mixture)[0]
    mask = X == 0
    
    E_matrix = post @ mixture.mu  # Matrix with expected values for all points
    XX[mask] = E_matrix[mask]  # Update where there are missing values
    return XX

def run(X: np.ndarray, mixture: GaussianMixture,
            post: np.ndarray) -> Tuple[GaussianMixture, np.ndarray, float]:
        """Runs the mixture model
    
        Args:
            X: (n, d) array holding the data
            post: (n, K) array holding the soft counts
                for all components for all examples
    
        Returns:
            GaussianMixture: the new gaussian mixture
            np.ndarray: (n, K) array holding the soft counts
                for all components for all examples
            float: log-likelihood of the current assignment
        """
        old = None
        new = None
        while old is None or new - old > abs(new)*1e-6:
            
            old=new
            post , new = estep(X,mixture)
            mixture = mstep(X,post,mixture)         

        return (mixture, post, new)
    
def super_run(X,K,n_seeds):
        """Executes several runs with different seed values
    
        Args:
            X: (n, d) array holding the data
            K: int, number of clusters
            n_seeds: integer, used as range in for loop
    
        Returns:
            GaussianMixture: best performing GaussianMixture
            LL: float, log likelihood with best GaussianMixture
        """
        LL = []
        GM = []
        for seed in range (n_seeds):

            X_init = init(X,K,seed)
            run_result = run(X,X_init[0],X_init[1]) 
            GM += [run_result[0]]
            LL += [run_result[2]] #[seed]
            print(f'LL for K = {K} and seed = {seed}, \n{LL[seed]}')
        maxLL = max(LL) ; best_GM = GM[LL.index(maxLL)]


We proceed to load the dataset and calculate the best performing Gaussian Mixture. We shall use 12 clusters and 5 seeds.
(Note that we know that this is the optimal solution, We would iterate over several cluster numbers but it would be too long).

In [168]:
X = np.loadtxt('netflix_incomplete.txt')
best_GM=super_run(X,12,5)

<ipython-input-166-02c00728c08f>:61: RuntimeWarning: invalid value encountered in true_divide
  mu_hat = np.sum(post[:,[j]]*X,axis = 0)/np.sum(ind*post[:,[j]],axis = 0)


LL for K = 12 and seed = 0, 
-1399803.0466569131
LL for K = 12 and seed = 1, 
-1390234.4223469417
LL for K = 12 and seed = 2, 
-1416862.401151279
LL for K = 12 and seed = 3, 
-1393521.3929897777
LL for K = 12 and seed = 4, 
-1416733.8083763549


In [171]:
X_pred = fill_matrix(W,Best_GM[0]);
X_gold = np.loadtxt('netflix_complete.txt')
print( f'The RMSE between predicted and true values is {np.round(rmse(X_pred,X_gold),6)}')

The RMSE between predicted and true values is 0.480491


For assessment purposes, test incomplete, test complete and test solutions can be used to verify that the algorithm works as expected.

#### And that's all!
### Thank you for reading this far, I hope you've  enjoyed the exercise!